## Implement Dense Network 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 10

## DenseUnit 

In [14]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        self.concat = tf.keras.layers.Concatenate()
        
    def call(self, x, training=False, mask=None): # x: (Batch, H, W, Ch_in)
        h = self.bn(x, training)
        h = tf.nn.relu(h)
        h = self.conv(h) # h: (Batch, H, W, filter_out)
        return self.concat([x, h]) # (Batch, H, W, (Ch_in + filter_out))

## DenseLayer

In [15]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = []
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))
            
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training)
        return x

## TransitionLayer
- To prevent the huge growth of DenseLayer (with growth rate), TransitionLayer maintain the architecture with Pooling

In [16]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.pool = tf.keras.layers.MaxPool2D()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        return self.pool(x)

## Model

In [17]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x8
        self.dl1 = DenseLayer(2, 4, (3, 3)) # 28x28x16
        self.tr1 = TransitionLayer(16, (3, 3)) # 14x14x16
        
        self.dl2 = DenseLayer(2, 8, (3, 3)) # 14x14x32
        self.tr2 = TransitionLayer(32, (3, 3)) # 7x32
        
        self.dl3 = DenseLayer(2, 16, (3, 3)) # 7x7x64
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.dl1(x, training)
        x = self.tr1(x)
        
        x = self.dl2(x, training)
        x = self.tr2(x)
        
        x = self.dl3(x, training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## Train/Test Loop

In [18]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## Dataset

In [19]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Model Define

In [21]:
# Create a model
model = DenseNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance parameter
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [24]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1, 
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

Epoch 1, Loss: 0.07388606667518616, Accuracy: 97.83499908447266, Test Loss: 0.06060953810811043, Test Accuracy: 98.5199966430664
Epoch 2, Loss: 0.06552138179540634, Accuracy: 98.1050033569336, Test Loss: 0.08823598921298981, Test Accuracy: 97.97999572753906
Epoch 3, Loss: 0.05958382040262222, Accuracy: 98.29033660888672, Test Loss: 0.08642532676458359, Test Accuracy: 97.98999786376953
Epoch 4, Loss: 0.054806891828775406, Accuracy: 98.43500518798828, Test Loss: 0.07911261916160583, Test Accuracy: 98.15499877929688
Epoch 5, Loss: 0.05093156546354294, Accuracy: 98.5528564453125, Test Loss: 0.07734973728656769, Test Accuracy: 98.26399993896484
Epoch 6, Loss: 0.04785939306020737, Accuracy: 98.64812469482422, Test Loss: 0.07252150774002075, Test Accuracy: 98.36166381835938
Epoch 7, Loss: 0.04537026956677437, Accuracy: 98.73092651367188, Test Loss: 0.06984727829694748, Test Accuracy: 98.41714477539062
Epoch 8, Loss: 0.043192509561777115, Accuracy: 98.79766845703125, Test Loss: 0.0693260058760